In [412]:
import simpy
import random
import numpy as np

In [413]:
class Servers:
    def __init__(self, env, mu, num_servers):
        self.mu = mu
        self.server = simpy.Resource(env, capacity=num_servers)

def task(name, env, servers, waiting_times, print_tasks):

    # A task arrives at the server
    start_time = env.now
    if print_tasks:
        print(f'Task {name} arriving at {start_time}')

    # Request task to server
    with servers.server.request() as req:

        # Once there is availability at the server, initiate task
        yield req
        end_time = env.now
        if print_tasks:
            print(f'Server starts processing {name} at {end_time}')

        # Calculate time at server by Poisson process, not sure if necessary
        processing_time = max(0, int(random.normalvariate(1 / servers.mu, 1)))
        yield env.timeout(processing_time)

        if print_tasks:
            print(f'Server done processing {name} at {env.now}')

        waiting_times.append(end_time - start_time)


def task_generator(env, servers, N, lambda_, waiting_times, print_tasks=False):

    # Generate N tasks
    for i in range(N):

        # Create task
        env.process(task(i, env, servers, waiting_times, print_tasks))

        # Calculate time until next task by Poisson process, not sure if necessary
        next_arrival = max(0, int(random.normalvariate(1 / lambda_, 1)))
        yield env.timeout(next_arrival)


In [414]:
num_servers = [1, 2, 4]
mu = 0.2
lambda_ = 0.15
N = 100000
tmax = 2000000

for n in num_servers:
    env = simpy.Environment()
    servers = Servers(env, mu, n)
    waiting_times = []

    car_gen = env.process(task_generator(env, servers, N, lambda_*n, waiting_times))
    env.run(tmax)

    waiting_times_array = np.array(waiting_times)
    average_waiting_time = np.mean(waiting_times_array)
    std_dev_waiting_time = np.std(waiting_times_array)

    print(f"{n} servers:")
    print(f"Average Waiting Time: {average_waiting_time}")
    print(f"Standard Deviation of Waiting Time: {std_dev_waiting_time} \n")

1 servers:
Average Waiting Time: 0.1132
Standard Deviation of Waiting Time: 0.4211481449561425 

2 servers:
Average Waiting Time: 0.48316
Standard Deviation of Waiting Time: 1.0161576720174876 

4 servers:
Average Waiting Time: 4.12448
Standard Deviation of Waiting Time: 4.869259156134535 

